## Notebook to test the c find and move functionality in pynetdicom

In [7]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from pydicom.dataset import Dataset
from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelFind

In [9]:
from pynetdicom import (
    AE, evt, build_role,
    PYNETDICOM_IMPLEMENTATION_UID,
    PYNETDICOM_IMPLEMENTATION_VERSION,
    StoragePresentationContexts
)
from pynetdicom.sop_class import (
    PatientRootQueryRetrieveInformationModelMove,
    CTImageStorage,
    ComputedRadiographyImageStorage
)

In [10]:
import sys

In [11]:
sys.path.append("lib/")

In [12]:
from Utils import *

## Start the SCP, for c - Move

In [13]:
handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE(ae_title = 'RBTEST')

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)

# Add the supported context
ae.supported_contexts = StoragePresentationContexts

# start the service class provider specifying the port
scp = ae.start_server(('', 4284), block=False, evt_handlers=handlers)

## Implement the c - Find

In [14]:
# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelFind)

# Create our Identifier (query) dataset
ds = Dataset()
ds.PatientID = "*"
ds.AccessionNumber = "934531"
ds.StudyInstanceUID = '*'
ds.QueryRetrieveLevel = 'PATIENT'
ds.SeriesInstanceUID = "*"
#ds.ReferencedInstanceSequence = "*"

In [24]:
# Associate with peer AE at IP 10.200.200.201 and port 104
assoc = ae.associate('10.200.200.201', port = 104)

if assoc.is_established:
    # Use the C-FIND service to send the identifier
    responses = assoc.send_c_find(ds, query_model = PatientRootQueryRetrieveInformationModelFind)
    for (status, identifier) in responses:
        if status:
            print('C-FIND query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then identifier is the C-FIND response
            if status.Status in (0xFF00, 0xFF01):
                ds = identifier
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

     # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0010, 0020) Patient ID                          LO: '4482356'
(0020, 000d) Study Instance UID                  UI: 1.2.840.113745.101000.1186002.40721.8111.13846623
(0020, 000e) Series Instance UID                 UI: 1.2.276.0.7230010.3.1.3.0.32423.1424660650.923422
C-FIND query status: 0x0000


## Implement C - move with SCP

In [16]:
from pydicom.dataset import Dataset

from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelMove

In [17]:
ds.QueryRetrieveLevel = 'PATIENT'
delattr(ds, 'AccessionNumber')
orig_ds = ds

In [18]:
ds

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0010, 0020) Patient ID                          LO: '4482356'
(0020, 000d) Study Instance UID                  UI: 1.2.840.113745.101000.1186002.40721.8111.13846623
(0020, 000e) Series Instance UID                 UI: 1.2.276.0.7230010.3.1.3.0.32423.1424660650.923422

In [21]:
# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level
ds = orig_ds

# Associate with peer AE at IP 10.200.200.201 and port 104
assoc = ae.associate('10.200.200.201', 104)

if assoc.is_established:
    # Use the C-MOVE service to send the identifier. RBTEST specifies the AE title of the Storage SCP
    # The event handler specified when initializing the SCP earlier determines what happens with the identifier
    responses = assoc.send_c_move(ds, 'RBTEST', PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then `identifier` is the C-GET response
            if status.Status in (0xFF00, 0xFF01):
                ct_ds = identifier
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')
    
#scp.shutdown()

C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0x0000


In [22]:
scp.shutdown()